In [204]:
import os
import itertools
import collections
import numpy as np
import pandas as pd

import scipy.optimize as sco

In [205]:
code_list=['2914', '3382', '4063', '4452', '4502', '4503',
                    '4568', '6098', '6501',  '6758', '6861', '6954',
                    '6981', '7203', '7267',  '7751', '7974', '8031',
                    '8058', '8306', '8316',  '8411', '8766', '8802',
                    '9020', '9022', '9432',  '9433', '9437', '9984']

In [218]:
for i, code in enumerate(code_list):
    tmp = pd.read_csv('../data/stock_price/' + str(code) + '.csv', index_col=0)
    tmp.index = pd.to_datetime(tmp['date'])
    tmp = tmp[((tmp.index.year >= 2013) & (tmp.index.year <= 2018))]
    tmp = tmp.drop(['date'], axis=1)
    if i == 0:
#         df2 = tmp.drop(['date'], axis=1)
        df2 = df2.rename({'adj_close': str(code)}, axis=1)
    else:
        df2 = pd.concat([df2, tmp], axis=1, join='outer')

df6 = pd.read_csv('../data/nikkei_225.csv')
df6.index = pd.to_datetime(df6['date'])
df6 = df6.drop(['date'], axis=1)

df2 = df2.join(df6)

In [219]:
df2

,2914,3382,4063,4452,4502,4503,4568,adj_close,adj_close,adj_close,...,adj_close,adj_close,adj_close,adj_close,adj_close,adj_close,adj_close,adj_close,adj_close,value
date,,,,,,,,,,,,,,,,,,,,,
2013-01-04,2521.0,2491.0,5400.0,2290.0,3975.0,806.0,1361.0,2491.0,5400.0,2290.0,...,163.0,2482.0,2111.0,5600.0,7100.0,1842.5,1031.7,1270.0,1562.5,10688.11
2013-01-07,2593.0,2617.0,5270.0,2323.0,3990.0,810.0,1369.0,2617.0,5270.0,2323.0,...,162.0,2471.0,2100.0,5650.0,7130.0,1855.0,1028.3,1268.0,1532.5,10599.01
2013-01-08,2608.0,2602.0,5290.0,2331.0,4005.0,814.0,1362.0,2602.0,5290.0,2331.0,...,161.0,2417.0,2006.0,5730.0,7240.0,1847.5,1038.3,1272.0,1552.5,10508.06
2013-01-09,2571.0,2649.0,5270.0,2333.0,4025.0,814.0,1375.0,2649.0,5270.0,2333.0,...,163.0,2457.0,2072.0,5720.0,7190.0,1840.0,1051.7,1260.0,1520.0,10578.57
2013-01-10,2626.0,2620.0,5380.0,2353.0,4140.0,823.0,1392.0,2620.0,5380.0,2353.0,...,167.0,2582.0,2079.0,5750.0,7150.0,1855.0,1033.3,1269.0,1520.0,10652.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-21,2681.5,4764.0,8555.0,8073.0,3835.0,1462.0,3553.0,4764.0,8555.0,8073.0,...,170.8,5198.0,1740.0,9480.0,22200.0,4408.0,2598.5,2441.0,3810.5,20166.19
2018-12-25,2587.0,4588.0,8056.0,7702.0,3640.0,1374.5,3286.0,4588.0,8056.0,7702.0,...,165.8,5030.0,1677.0,9153.0,21570.0,4245.0,2464.0,2341.5,3521.5,19155.74
2018-12-26,2511.5,4654.0,8147.0,7717.0,3718.0,1393.5,3339.0,4654.0,8147.0,7717.0,...,166.5,5006.0,1684.0,9265.0,21835.0,4292.0,2516.0,2388.5,3473.5,19327.06


In [207]:
def calc_act_return(x):
    print(x.shape)
    x = x.pct_change(1).shift(-1)*100
    return x.drop(['value'], axis=1) - x['value']

In [208]:
df2

,2914,3382,4063,4452,4502,4503,4568
date,,,,,,,
2013-01-04,2521.0,2491.0,5400.0,2290.0,3975.0,806.0,1361.0
2013-01-07,2593.0,2617.0,5270.0,2323.0,3990.0,810.0,1369.0
2013-01-08,2608.0,2602.0,5290.0,2331.0,4005.0,814.0,1362.0
2013-01-09,2571.0,2649.0,5270.0,2333.0,4025.0,814.0,1375.0
2013-01-10,2626.0,2620.0,5380.0,2353.0,4140.0,823.0,1392.0
...,...,...,...,...,...,...,...
2018-12-21,2681.5,4764.0,8555.0,8073.0,3835.0,1462.0,3553.0
2018-12-25,2587.0,4588.0,8056.0,7702.0,3640.0,1374.5,3286.0
2018-12-26,2511.5,4654.0,8147.0,7717.0,3718.0,1393.5,3339.0


In [193]:
tmp = df2.pct_change(1).shift(-1)*100
df3 = tmp.sub(tmp['value'], axis='index')

# B&H

In [200]:
train = df2[df2.index.year <= 2016]
test = df2[df2.index.year > 2016]

,2914,3382,4063,4452,4502,4503,4568,6098,6501,6758,...,8411,8766,8802,9020,9022,9432,9433,9437,9984,value
date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,4328.0,5395.0,6483.0,6012.0,5917.0,1685.5,2427.5,1170.0,3383.0,2957.0,...,239.7,4601.0,2465.5,11125.0,21010.0,4714.0,3036.0,2426.0,2996.5,18450.98
2016-01-05,4246.0,5387.0,6474.0,5928.0,5853.0,1663.0,2438.0,1193.3,3366.5,2962.5,...,239.5,4515.0,2449.0,11205.0,21150.0,4815.0,3087.0,2491.0,2984.5,18374.00
2016-01-06,4268.0,5356.0,6413.0,5944.0,5869.0,1674.5,2439.0,1175.0,3259.5,2897.5,...,236.5,4439.0,2451.5,11135.0,20985.0,4879.0,3050.0,2461.0,2944.5,18191.32
2016-01-07,4243.0,5312.0,6207.0,5834.0,5853.0,1679.0,2442.5,1173.3,3173.5,2796.5,...,231.0,4391.0,2386.0,11050.0,20545.0,4797.0,3009.0,2438.0,2875.5,17767.34
2016-01-08,4193.0,5141.0,6183.0,5727.0,5787.0,1677.0,2417.5,1168.3,3165.5,2824.5,...,229.4,4335.0,2346.5,10875.0,20405.0,4778.0,3005.0,2430.5,2847.5,17697.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-26,3935.0,4512.0,9281.0,5626.0,4855.0,1634.0,2390.0,1551.7,3195.0,3333.0,...,214.9,4863.0,2364.5,10260.0,19530.0,4946.0,2950.5,2672.5,3923.5,19396.64
2016-12-27,3922.0,4487.0,9234.0,5651.0,4851.0,1635.0,2380.0,1551.7,3183.0,3324.0,...,215.7,4859.0,2332.0,10250.0,19485.0,4927.0,2974.0,2660.5,3934.0,19403.06
2016-12-28,3839.0,4485.0,9299.0,5558.0,4849.0,1620.5,2387.0,1566.7,3224.5,3338.0,...,214.5,4826.0,2334.5,10265.0,19480.0,4929.0,2963.5,2670.5,3929.0,19401.72


In [203]:
train

,2914,3382,4063,4452,4502,4503,4568,6098,6501,6758,...,8411,8766,8802,9020,9022,9432,9433,9437,9984,value
date,,,,,,,,,,,,,,,,,,,,,
2016-01-04,4328.0,5395.0,6483.0,6012.0,5917.0,1685.5,2427.5,1170.0,3383.0,2957.0,...,239.7,4601.0,2465.5,11125.0,21010.0,4714.0,3036.0,2426.0,2996.5,18450.98
2016-01-05,4246.0,5387.0,6474.0,5928.0,5853.0,1663.0,2438.0,1193.3,3366.5,2962.5,...,239.5,4515.0,2449.0,11205.0,21150.0,4815.0,3087.0,2491.0,2984.5,18374.00
2016-01-06,4268.0,5356.0,6413.0,5944.0,5869.0,1674.5,2439.0,1175.0,3259.5,2897.5,...,236.5,4439.0,2451.5,11135.0,20985.0,4879.0,3050.0,2461.0,2944.5,18191.32
2016-01-07,4243.0,5312.0,6207.0,5834.0,5853.0,1679.0,2442.5,1173.3,3173.5,2796.5,...,231.0,4391.0,2386.0,11050.0,20545.0,4797.0,3009.0,2438.0,2875.5,17767.34
2016-01-08,4193.0,5141.0,6183.0,5727.0,5787.0,1677.0,2417.5,1168.3,3165.5,2824.5,...,229.4,4335.0,2346.5,10875.0,20405.0,4778.0,3005.0,2430.5,2847.5,17697.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-26,3935.0,4512.0,9281.0,5626.0,4855.0,1634.0,2390.0,1551.7,3195.0,3333.0,...,214.9,4863.0,2364.5,10260.0,19530.0,4946.0,2950.5,2672.5,3923.5,19396.64
2016-12-27,3922.0,4487.0,9234.0,5651.0,4851.0,1635.0,2380.0,1551.7,3183.0,3324.0,...,215.7,4859.0,2332.0,10250.0,19485.0,4927.0,2974.0,2660.5,3934.0,19403.06
2016-12-28,3839.0,4485.0,9299.0,5558.0,4849.0,1620.5,2387.0,1566.7,3224.5,3338.0,...,214.5,4826.0,2334.5,10265.0,19480.0,4929.0,2963.5,2670.5,3929.0,19401.72


# Markowitz